###############################
##    뉴스 기사 테마 추천   ##
###############################

0. 라이브러리 임포트

In [1]:
# -*- coding: utf-8 -*-
import numpy as np
from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import os
import pickle
import re
import itertools

c:\Users\G-10\anaconda3\envs\tensor\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


1. 뉴스기사 전처리

In [2]:
##      뉴스기사
article = """
의료현장에서 사용하기 위한 인공지능(AI) 시스템의 성능이 충분히 검증되지 못하고 있다는 분석이 나왔다.

시스템의 정확도를 면밀하게 검증하기 위해선 지금보다 더 많은 단계의 시험을 거쳐야 한다는 지적이다. 작은 이상 증상에도 민감하게 반응하는 등 AI의 정밀도를 제고해야 의료 현장에서 실제로 사용이 늘어날 것이라는 분석이 나온다.

22일 국제학술지 네이처에 따르면 의료·AI 전문가들은 현재 의료현장에서 사용되고 있는 A I시스템에 대해 이같은 분석을 제시했다.

앞서 지난해 영국 버밍엄대 연구팀이 국제학술지 '네이처 커뮤니케이션즈'에 발표한 논문에 따르면 AI 시스템이 의료현장에서 사용하기 적절한지 확인하기 위해선 여러 단계의 검증 실험이 이뤄져야 한다. 특히 연구 대상자를 무작위로 두 그룹 이상으로 나눠 효과를 확인하는 무작위 대조 시험은 필수적이다.

그러나 2020년부터 2022년 사이 의료 AI 시스템 개입의 효과를 확인한 연구에서 무작위 대조 시험이 이뤄진 경우는 65건에 불과했다. 같은 기간 미국 식품의약국(FDA)은 의료기관에서 사용할 수 있는 수백 대의 AI 기반 의료기기를 승인했다. 데이비드 오우양 미국 시더스시나이 메디컬센터 교수는 "의료기관들은 임상적으로 검증되지 않은 채 승인된 AI 시스템을 사용하고 있다"고 전했다.

의료현장에서 축적된 데이터만을 검증하는 방식도 부족하다는 지적이다. 앞서 캐나다에서 이뤄진 한 연구에선 응급 환자에게 필요한 진단을 정확히 예측하는 AI 시스템을 검증하기 위해 7만7000건의 응급실 의료 데이터를 분석한 뒤, 이 시스템이 응급실 의료처치의 효율을 22.3% 높일 수 있다는 결과를 발표했다. 하지만 이러한 분석은 AI 시스템의 성능을 검증하기 위한 첫 단계에 불과하다는 것이다.

전문가들은 AI 시스템의 효용성을 정확히 시험하기 위해선 후향적 연구가 아닌 전향적 연구가 이뤄져야 한다고 말한다. 이미 알고 있는 과거의 결과를 분석해 인과관계를 관찰하는 후향적 연구는 연구 대상의 돌발적인 변수를 예측하기 어렵다는 것이다. 캐나다에서 진행된 응급실 연구 또한 대량의 데이터를 분석했지만 후향적 연구인 만큼 한계점이 존재한다는 게 전문가들의 이야기다.

의료진들이 AI 시스템을 신뢰하지 않는다는 지적도 제기된다. 상대적으로 가벼운 증상에도 작동하는 AI의 알람음은 의료진들에게 '경보 피로'를 일으킨다는 것이다. 데이비드 러쉬로우 미국 메이오클리닉 교수는 "바쁜 일선 의사들은 환자들이 위험에 처할 수 있는 상태에 대한 경고에 하루에도 여러 번 노출된다"며 AI시스템의 경고 정확성이 담보되지 않는다면 의사들은 경보음에 귀를 기울이지 않을 것이라 말했다.

AI 시스템이 정확도를 높이기 위해선 다양한 인종을 대상으로 한 연구가 확장돼야 한다는 분석도 나온다. 예를 들어 앞서 구글 헬스가 미국 환자들을 대상으로 연구해 개발한 당뇨병성 망막증 감지 AI 알고리즘은 이론적으로 정확했지만 실제 이 시스템이 태국에서 사용됐을 때는 정확도가 크게 떨어졌다.

일부 전문가들은 현시점에서 의료 AI 시스템의 성능을 검증하기 위해 가장 좋은 방법은 각 의료기관이 도입에 앞서 자체 실험을 진행하는 것이라 말했다. 메이요클리닉의 AI검증 연구 프로그램을 이끄는 쇼사나 오버가아드는 "메이요클리닉과 연계된 지역 사회 보건 시설에서 표준화된 방식으로 AI 시스템의 성능을 확인하고 있다"고 전했다.
"""
article = article.replace('\n', '')
split_article = re.split(r'(\s*[a-zA-Z]+(?:[^\w\s]+[a-zA-Z]+)*\s*)', article)       #   한글과 영어 나누기
split_article = [part.strip() for part in split_article if part.strip()]            #   불필요한 공백 지우기

okt = Okt()                         #   형태소 분석기 Okt
nouns=[]
for sentence in split_article:
    sen = okt.nouns(sentence)
    if len(sen) != 0:               #   명사를 리스트에 추가
        nouns.extend(sen)
    else:
        nouns.append(sentence)      #   영어도 리스트에 추가(형태소 분석 후 길이가 0이면 영어로 판단)
    # nouns.append(sentence)

text = ' '.join(nouns)              #   리스트를 하나의 문자열로

print(f'전처리 완료 후 뉴스 기사 : {text}')

전처리 완료 후 뉴스 기사 : 의료 현장 사용 위 인공 지능 AI 시스템 성능 검증 못 분석 시스템 정확도 검증 위해 선 지금 더 단계 시험 지적 이상 증상 반응 등 AI 의 정밀도 고해 의료 현장 실제 사용 것 분석 국제 학술지 네이처 의료 AI 전문가 현재 의료 현장 사용 A I 시스템 대해 이 분석 제시 앞서 지난해 영국 버밍엄 팀 국제 학술지 네이처 커뮤니케이션 발표 논문 AI 시스템 의료 현장 사용 확인 위해 선 여러 단계 검증 실험 연구 대상자 무작위 두 그룹 이상 효과 확인 무작위 대조 시험 필수 사이 의료 AI 시스템 개입 효과 확인 연구 무작위 대조 시험 경우 건 기간 미국 식품 약국 FDA 은 의료 기관 사용 수 수백 대의 AI 기반 의료기기 승인 데이비드 오우 미국 시더 스시 나이 컬 센터 교수 의료 기관 임 상적 검증 채 승인 AI 시스템 사용 고 전 의료 현장 축적 데이터 검증 방식 지적 앞서 캐나다 연구 응급 환자 진단 예측 AI 시스템 검증 위해 건의 응급실 의료 데이터 분석 뒤 이 시스템 응급실 의료 처치 효율 수 결과 발표 분석 AI 시스템 성능 검증 위 첫 단계 것 전문가 AI 시스템 효용 시험 위해 선 후향 연구 전향 연구 말 알 과거 결과 분석 인과관계 관찰 후향 연구 연구 대상 돌발 변수 예측 것 캐나다 진행 응급실 연구 또한 대량 데이터 분석 후향 연구 만큼 한계 점 존재 게 전문가 이야기 의료 진 AI 시스템 신뢰 지적도 제기 상대 증상 작동 AI 의 알람 음 의료 진 경보 피로 를 것 데이비드 러쉬 로우 미국 메이 오클리 닉 교수 일선 의사 환자 위험 처 수 상태 대한 경고 하루 여러 번 노출 며 AI 시스템 경고 정확성 담보 의사 경보 음 귀 것 말 AI 시스템 정확도 위해 선 인종 대상 연구 확장 분석 예 앞서 구글 헬스 미국 환자 대상 연구 개발 당뇨병 망막 증 감지 AI 알고리즘 이론 실제 이 시스템 태국 사용 때 정확도 크게 일부 전문가 시점 의료 AI 시스템 성능 검증 위해 가장 방법 각 의료 기관 도입 앞

2. N-Gram 추출 및 SBERT

In [3]:
n_gram_range = (1,2)
word_vectorize = CountVectorizer(ngram_range=n_gram_range).fit([text])   #   1 ~ 2 글자로 N-Gram 생성
n_gram_words = word_vectorize.get_feature_names_out()

model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')  #, clean_up_tokenization_spaces=False
article_embedding = model.encode([article], )
n_gram_embeddings = model.encode(n_gram_words)

print(f'N-Gram의 길이 : {len(n_gram_words)}')
print(f'N-Gram 예시 : {n_gram_words[:5]}')

c:\Users\G-10\anaconda3\envs\tensor\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


N-Gram의 길이 : 398
N-Gram 예시 : ['ai' 'ai 검증' 'ai 기반' 'ai 시스템' 'ai 알고리즘']


3. 코사인 거리 유사도 기반 키워드 추출

In [4]:
def max_sum_sim(article_embedding, n_gram_embeddings, n_gram_words, top_n, variety):
    distances = cosine_similarity(article_embedding, n_gram_embeddings)                     #   뉴스와 N-Gram 사이의 유사도
    distances_candidates = cosine_similarity(n_gram_embeddings, n_gram_embeddings)          #   N-Gram들 사이의 유사도

    words_idx = list(distances.argsort()[0][-variety:])                                     #   다양성 설정
    words_vals = [n_gram_words[index] for index in words_idx]
    distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

    #   그 중 유사도가 가장 낮은 조합 찾기
    min_sim = np.inf
    candidate = None
    for combination in itertools.combinations(range(len(words_idx)), top_n):
        sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
        if sim < min_sim:
            candidate = combination
            min_sim = sim

    return [words_vals[idx] for idx in candidate]

news_keywords = max_sum_sim(article_embedding, n_gram_embeddings, n_gram_words, top_n=5, variety=10)
print(f'추출된 키워드 : {news_keywords}')

추출된 키워드 : ['환자 진단', '의료기기 승인', '의료 데이터', '실험 연구', '연구 확장']


4. 테마별 임베딩 모두 불러오기

In [6]:
directory_path = 'D:/TJ_FInal_Project/KDJ/News_Keyword_Theme_Recommend.py/embeddings'
all_embeddings = []
file_names = []

for filename in os.listdir(directory_path):
    # 디렉토리 내에 모든 피클 파일 가져오기
    if filename.endswith('.pkl'):
        file_path = os.path.join(directory_path, filename)
        
        with open(file_path, 'rb') as f:
            embedding = pickle.load(f)
            all_embeddings.append(embedding)

        # 테마 이름 저장하기
        name_without_extension = filename[:-4]
        file_names.append(name_without_extension)

print(f'가져온 테마 임베딩 개수 : {len(all_embeddings)}개')
print(f'가져온 테마 임베딩 개수 : {len(file_names)}개')
print(f'테마이름 예시 : {file_names[:5]}')

가져온 테마 임베딩 개수 : 251개
가져온 테마 임베딩 개수 : 251개
테마이름 예시 : ['2차전지 장비', '2차전지(소재부품)', '2차전지', '3D프린터', '5G']


5. 코사인 유사도 기반 테마 추천

In [8]:
theme_score = []
news_keywords_embeddings = model.encode([news_keywords])
for i in range(0, len(all_embeddings)):
  distances = cosine_similarity(all_embeddings[i], article_embedding) #article_embedding
  number = distances.item()
  theme_score.append(number)
  
how_many_theme = 5
top_indices = sorted(range(len(theme_score)), key=lambda i: theme_score[i], reverse=True)[:how_many_theme]

with open('D:/TJ_FInal_Project/KDJ/News_Keyword_Theme_Recommend.py/data/theme_dict.pkl', 'rb') as f:
    theme_dict = pickle.load(f)
print(f'뉴스 키워드 5개 : {news_keywords}\n')
for i in range(0,how_many_theme):
  index = top_indices[i]
  theme_recommend = file_names[index]
  print(f'추천 테마 : {theme_recommend}\n테마주 : {theme_dict[theme_recommend]}\n')

뉴스 키워드 5개 : ['환자 진단', '의료기기 승인', '의료 데이터', '실험 연구', '연구 확장']

추천 테마 : 의료AI
테마주 : ['루닛', '뷰노', '셀바스AI', '랩지노믹스', '제이엘케이', '셀바스헬스케어', '딥노이드', '파로스아이바이오', '인성정보', '코어라인소프트', '이지케어텍', '신테카바이오', '토마토시스템', '드림씨아이에스', '셀루메드', '노을', '디엔에이링크', '라이프시맨틱스', '모아데이타', '젠큐릭스', '비스토스', '씨유박스', '엔젠바이오']

추천 테마 : 줄기세포
테마주 : ['차바이오텍', '네이처셀', '파미셀', '코아스템켐온', '제일약품', '메디포스트', '동구바이오제약', '테고사이언스', '안트로젠', '스피어파워', '엠젠솔루션', '에스씨엠생명과학', '바이온']

추천 테마 : 혈장치료
테마주 : ['신풍제약', '시노펙스', '넵튠', '레몬', '에코볼트']

추천 테마 : 초전도체
테마주 : ['LS ELECTRIC', '신성델타테크', '일진전기', 'LS에코에너지', '고려제강', '한양이엔지', '파워로직스', '비츠로테크', '원익피앤이', '씨씨에스', '인지컨트롤스', '덕성', '서남', '모비스', '탑엔지니어링', '아모텍', '인지디스플레', '아센디오']

추천 테마 : 비대면 진료(AI 진료)
테마주 : ['루닛', '유비케어', '제이엘케이', '딥노이드', '인성정보', '인피니트헬스케어', '이지케어텍', '비트컴퓨터', '케어랩스', '라이프시맨틱스', '소프트센']

